## Prompt engieneering

This script tries to highlight the need for prompt engineering. Specifically it tries to find the line which distincts 


The LLM will receive three different questions which increase in complexity:

1. What is 1000*55 (simple and deterministic)
2. Who is the most powerful character in Star Wars? (simple but subjective, equiring reasoning)
3. What is the effect of wind turbines on property values?

Two different LLMs are tested:
1. Gemma3:1b "Gemma is a lightweight, family of models [...], they excel in tasks like question answering, summarization, and reasoning, while their compact design allows deployment on resource-limited devices." https://ollama.com/library/gemma3:1b

2.




In [12]:
# %pip install ollama
from ollama import chat

deterministic_content = 'What is 1000*55?'
ambiguous_content = 'Who is the most powerful character in Star Wars?'
complex_content = 'What is the mean effect of wind turbines on property values in percent?'

In [18]:
# Example of a simple question
response_easy = chat(model='gemma3:1b', messages=[
  {    'role': 'user', 'content': deterministic_content, }])

# Example of an ambiguous question
response_ambiguous = chat(model='gemma3:1b', messages=[
  {    'role': 'user', 'content': ambiguous_content, },])
# Example of a complex question
response_complex = chat(model='gemma3:1b', messages=[
    {   'role': 'user', 'content': complex_content,}])


In [19]:
print(response_easy.message.content)

1000 * 55 = 55000

Therefore, 1000 * 55 = **55000**



In [20]:
print(response_ambiguous.message.content)

Okay, this is a *really* hotly debated question in the Star Wars fandom, and there's no single, definitive answer! However, **Luke Skywalker** is widely considered the *most* powerful character in the Star Wars universe, at least in the sense of having the *potential* for immense power. 

Here's a breakdown of why, and why it’s so complex:

**Luke Skywalker - The Top Contender**

* **The Force Channeling:** Luke possesses an extraordinarily high level of connection to the Force. He’s shown to have a significantly enhanced Force ability compared to most Force users.  His connection is so potent that he can even subtly influence the Force, allowing him to subtly manipulate events.
* **Force Potential:**  His Force potential is arguably the highest of any character in the original trilogy. He can perform feats of incredible strength, speed, and even manipulate energy.
* **Resilience & Adaptability:** Luke's training and connection to the Force have made him incredibly resilient. He can wi

In [21]:
print(response_complex.message.content)

Okay, let's break down the research and data surrounding the impact of wind turbines on property values. It's a complex and somewhat controversial topic, with findings varying significantly depending on the study, location, and methodology. **There's no single, universally agreed-upon "mean effect" percentage.** However, here’s a summary of what the research shows, broken down into key findings and estimates:

**1. The Core of the Debate: The "NIMBY" Effect (Noise and Aesthetic Concerns)**

The overwhelming majority of research points to a *negative* impact on property values, primarily driven by concerns about noise and visual impact (aesthetic) associated with wind turbines.  This is often referred to as the "NIMBY" (Not In My Backyard) effect.

**2. Estimated Impact Ranges (with caveats):**

* **Economically Significant:**  Most studies estimate that wind turbines can reduce property values by **5-15%** on average.  Some very large-scale studies suggest a lower impact – even as low 

In general the answer list quite fitting influence factors of wind turbines on propert values (distance, noise, visual impact) but hallucinates terribly when it comes to the exact numbers (stating 5-15% on average) and also with regard to the studies cited.

## Let's give the LLM a scientific article on the topic and ask it to answer only with the help of this 

In [1]:
#current directory
import os
print(os.getcwd())

/workspaces/research-agent


In [24]:
%pip install PyPDF2
import PyPDF2

with open("./Wind turbines and property values.pdf", "rb") as file:
    reader = PyPDF2.PdfReader(file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()

print(text)

Note: you may need to restart the kernel to use updated packages.
Vol.:(0123456789)Environmental and Resource Economics (2024) 87:1–43
https://doi.org/10.1007/s10640-023-00809-y
1 3
Wind Turbines and Property Values: A Meta‑Regression 
Analysis
Marvin Schütt1 
Accepted: 4 September 2023 / Published online: 7 December 2023 
© The Author(s) 2023
Abstract
A key concern for property owners about the set up of proximate wind turbines is the 
potential devaluation of their property. However, there is no consensus in the empirical 
hedonic literature estimating this price-distance relationship. It remains unclear if the prox-
imity to wind turbines reduces, increases, or has no significant effect on property values. 
This article addresses this ambiguity, combining 720 estimates from 25 hedonic pricing 
studies in a first comprehensive meta-analysis on this topic. Using Bayesian model averag-
ing techniques and novel publication bias correction methods, I calculate an average of 
the reported

In [29]:
developer_prompt = f"""
# Identity

You are a senior economist teaching students about the impact of wind turbines on property values. You are an expert in the field and can provide detailed explanations and analyses.

# Instructions

1. Only answer based on the provided 'scientific article'.
2. If there is no answer or not a definite answer to the question in the 'scientific article', say it so.
3. Write your answer in a concise and clear manner.

# 'Scientific Article':
{text}
"""


In [31]:
response_developer = chat(model='gemma3:1b', messages=[
    {   'role': 'system', 'content': developer_prompt, },
    {   'role': 'user', 'content': complex_content,}])  

print(response_developer.message.content)

Okay, let's break down the research and current understanding of the effect of wind turbines on property values. It's a complex and hotly debated topic, but here's a summary of what the research suggests, and a reasonable estimate of the "mean effect" percentage:

**1. The Research Landscape - Key Findings**

* **Mixed Results - No Simple Answer:**  The overwhelming consensus isn't that wind turbines *always* decrease property values. Instead, the impact is nuanced and varies significantly by location, turbine size, and other factors.
* **Small to Moderate Impact:** The vast majority of studies (around 70-85%) find that wind turbines *slightly* reduce property values.  The reduction is generally considered to be in the range of **0.5% to 2%**. Some studies even find a negative impact.
* **Location Matters Most:**  This is the *most* critical factor.  Studies consistently show a stronger negative effect in areas with:
    * **High turbine density:** More turbines per square mile.
    * 

#The result is still very much hallucinated. 
Let's try a an even more precise promt.

In [32]:
developer_prompt_extended = f"""
# Identity

Act as a senior economist teaching students about the impact of wind turbines on property values. 
# Instructions

1. ONLY answer based on the provided 'scientific article'.
2. If there is no answer or not a definite answer to the question in the 'scientific article', say it so.
3. Write your answer in a concise and clear manner.
4. Use the following format for your answer:
   - 'Answer': The answer to the question.
    - 'Explanation': A brief explanation of how you arrived at the answer.

# 'Scientific Article':
###
{text}
###
"""

In [33]:
response_developer_extended = chat(model='gemma3:1b', messages=[
    {   'role': 'system', 'content': developer_prompt_extended, },
    {   'role': 'user', 'content': complex_content,}])  

print(response_developer_extended.message.content)

Okay, let's break down the research and estimates on the impact of wind turbines on property values. It’s a surprisingly complex topic with varying results depending on the study, location, and methodology. Here's a summary of what the research suggests, and a general estimate of the percent impact:

**1. The Research Landscape – Key Findings**

* **Mixed Results - Most Studies Indicate a Small, but Significant, Increase:**  The overwhelming consensus is that wind turbines *do* have a statistically significant, but relatively small, positive impact on property values.  However, the magnitude of this impact is debated and varies considerably.
* **Range of Estimates:**  Studies have produced a range of estimates, with most placing the impact *between 2% and 8%* on average.  Some extremely large studies have even suggested a 10% or more effect.
* **Location Matters:**  This is the *biggest* factor.  Studies consistently show a *larger* impact in areas with high wind potential, established

## Still bad answer.

### Maybe try a better LLM:



In [34]:
response_developer_extended = chat(model='llama3.1:8b', messages=[
    {   'role': 'system', 'content': developer_prompt_extended, },
    {   'role': 'user', 'content': complex_content,}])  

print(response_developer_extended.message.content)

The impact of wind turbines on property values can vary greatly depending on several factors, including:

1. **Distance from homes**: Turbines located closer to homes tend to have a greater negative impact.
2. **Noise levels**: Noisy turbines can negatively affect property values more than quiet ones.
3. **Aesthetics**: The visual appeal of the turbines and their surroundings play a significant role in determining property value impacts.
4. **Local context**: Cultural, economic, and social factors specific to each community can influence how residents perceive wind farms.

Given these complexities, studies on this topic often produce varying results. Here are some general insights:

* A 2019 study by the Lawrence Berkeley National Laboratory found that:
	+ For every megawatt of installed capacity in a wind farm, property values decrease by approximately 1-3% within a radius of 1-2 miles.
	+ However, the overall impact on property values is relatively small, with an average effect size 

In [ ]:
FROM llama3.1:8b

# set the system message
SYSTEM """
Act as a senior economist teaching students about the impact of wind turbines on property values. 
# Instructions

1. ONLY answer based on the provided 'scientific article'.
2. If there is no answer or not a definite answer to the question in the 'scientific article', say it so.
3. Write your answer in a concise and clear manner.
4. Use the following format for your answer:
   - 'Answer': The answer to the question.
    - 'Explanation': A brief explanation of how you arrived at the answer.
"""

SyntaxError: invalid syntax (1720693705.py, line 1)